In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

import scipy.ndimage as ndi

from ImageD11.nbGui import nb_utils as utils
import ImageD11.sinograms.dataset
import numba

from ImageD11.grain import grain
from ImageD11 import unitcell
%matplotlib ipympl

In [ ]:
# USER: Pass path to dataset file

dset_file = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)

sample = ds.sample
dataset = ds.dsname
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# define your reference unit cell for RGB colour plotting

cf_2d = ds.get_cf_2d_from_disk()
cf_2d.parameters.loadparameters(ds.parfile)

cf_pars = cf_2d.parameters.get_parameters()

ref_ucell = ImageD11.unitcell.unitcell_from_parameters(cf_pars)

In [ ]:
class pbpmap:
    def __init__(self, fname):
        pbp_array = np.loadtxt(fname).T
        n = len(pbp_array[0])
        self.i = pbp_array[0].astype(int)
        self.i -= self.i.min()
        self.j = pbp_array[1].astype(int) 
        self.j -= self.j.min()
        self.n = pbp_array[2].astype(int)         # total peaks indexing with hkl==int with 0.03
        self.u = pbp_array[3].astype(int)         # unique (h,k,l) labels on indexed peaks
        self.NI = int(self.i.max() - self.i.min()) + 1
        self.NJ = int(self.j.max() - self.j.min()) + 1
        self.NY = max(self.NI,self.NJ)
        self.ubi = pbp_array[4:].astype(float)
        self.ubi.shape = 3,3,-1
        
    def choose_best(self, minpeaks=6):
        self.muniq, self.npks, self.ubibest = nb_choose_best( 
            self.i, self.j, 
            self.u, self.n, 
            self.NY, self.ubi, minpeaks)
    
@numba.njit
def nb_choose_best(i, j, u, n, NY, ubiar,
                minpeaks=6):
    # map of the unique scores
    uniq = np.ones((NY,NY), dtype='q')
    uniq.fill(minpeaks) # peak cutorr
    npk = np.zeros((NY,NY), dtype='q')
    ubi = np.zeros((NY,NY,3,3), dtype='d')
    ubi.fill(np.nan)
    for k in range(i.size):
        ip = i[k]
        jp = j[k]
#        if ip == 96 and jp == 510:
#            print(ip,jp,k, ubiar[:,:,k])
        if u[k] > uniq[ip,jp]:
            uniq[ip,jp] = u[k]
            npk[ip,jp] = n[k]
            for ii in range(3):
                for jj in range(3):          
                    ubi[ip,jp,ii,jj] = ubiar[ii,jj,k]
    return uniq, npk, ubi

def nb_inv(mats, imats):
    for i in range(mats.shape[0]):
        for j in range(mats.shape[1]):
            if np.isnan( mats[i,j,0,0] ):
                imats[i,j] = np.nan
            else:
                try:
                    imats[i,j] = np.linalg.inv( mats[i,j] )
                except:
                    print(i,j,mats[i,j])
                    break
                    imats[i,j] = 42.

In [ ]:
pbpfile = os.path.join(ds.analysispath, ds.dsname + '_pbp.txt')

pmap = pbpmap(pbpfile)

In [ ]:
fig, ax = plt.subplots()
ax.hist(pmap.u, bins=np.arange(0.5,np.max(pmap.u)+0.51,1))
ax.set_xlabel('Unique spots per pixel')
ax.set_ylabel('Count')
plt.show()

In [ ]:
pmap.choose_best(30)

In [ ]:
pmap.ubbest = np.zeros_like(pmap.ubibest)
nb_inv(pmap.ubibest, pmap.ubbest)

plot_mask = ~np.isnan(pmap.ubbest[:,:,0,0])
plot_mask.shape

pmap.ubplot = pmap.ubbest[plot_mask]

In [ ]:
# get a meta orientation for all the grains
pmap.meta_orien = ref_ucell.get_orix_orien(pmap.ubplot)

try:
    from orix.vector.vector3d import Vector3d
except ImportError:
    raise ImportError("Missing diffpy and/or orix, can't compute orix phase!")

axis = np.array([1., 0, 0])
ipf_direction = Vector3d(axis)

# get the RGB colours
pmap.rgb_x = ref_ucell.get_ipf_colour_from_orix_orien(pmap.meta_orien, axis=ipf_direction)
pmap.meta_orien.scatter("ipf", c=pmap.rgb_x, direction=ipf_direction, s=1)

axis = np.array([0., 1., 0])
ipf_direction = Vector3d(axis)
pmap.rgb_y = ref_ucell.get_ipf_colour_from_orix_orien(pmap.meta_orien, axis=ipf_direction)
pmap.meta_orien.scatter("ipf", c=pmap.rgb_y, direction=ipf_direction, s=1)

axis = np.array([0., 0., 1.])
ipf_direction = Vector3d(axis)
pmap.rgb_z = ref_ucell.get_ipf_colour_from_orix_orien(pmap.meta_orien, axis=ipf_direction)
pmap.meta_orien.scatter("ipf", c=pmap.rgb_z, direction=ipf_direction, s=1)

In [ ]:
# Fill some arrays with the colours

inds = np.mgrid[0:pmap.NY*pmap.NY][plot_mask.ravel()]
rgb = np.zeros( (3, pmap.NY,pmap.NY, 3 ))
rgb.shape = 3, -1, 3
rgb.fill(np.nan)
for k, axis in enumerate( ((1,0,0),(0,1,0),(0,0,1)) ):
    rgb[k, inds] = ref_ucell.get_ipf_colour( pmap.ubplot, axis )
rgb.shape = 3, pmap.NY, pmap.NY, 3

In [ ]:
fig, a = plt.subplots(2,2,sharex=True,sharey=True, figsize=(10,10), constrained_layout=True)
a[0][0].imshow(rgb[0], origin="lower")
a[0][0].set_title("IPF X")
a[0][1].imshow(rgb[1], origin="lower")
a[0][1].set_title("IPF Y")
a[1][0].imshow(rgb[2], origin="lower")
a[1][0].set_title("IPF Z")
r = pmap.muniq/pmap.muniq.max()
m = r > 0.02
a[1][1].imshow(np.where(m, r, np.nan), origin="lower")
a[1][1].set_title("Number of peaks")
# for ax in a.ravel():
#     ax.set(yticks=[], xticks=[])
#     ax.invert_yaxis()

plt.show()